# Instructions to get 1 game of SET scores

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.patches as patches
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.patches import Arrow

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from sklearn.cluster import KMeans
from collections import Counter
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.path as mplPath
from shapely.geometry import Polygon as PolygonArea
import matplotlib as mpl
import random
import os
from itertools import chain
from matplotlib.patches import Circle
from scipy.spatial import Voronoi,voronoi_plot_2d, ConvexHull
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.path as mplPath
from shapely.geometry import Polygon as PolygonArea

In [ ]:
#enter the week that the game is
week_number = 2

In [ ]:
#enter the plays.csv provided
plays = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/plays.csv',None)
#enter the tracking week you wish to look at
week_df = pd.read_csv(f'/content/drive/MyDrive/NFL DATA BOWL/tracking_week_{week_number}.csv',None)


In [ ]:
# Extract 'los_x' from 'week_df' where 'displayName' is 'football' and 'frameId' is 1
los_x = week_df[(week_df['displayName'] == 'football') & (week_df['frameId'] == 1)][['gameId', 'playId', 'x']].rename(columns={'x': 'los_x'})
# Merge 'los_x' back into 'week_df'
week_df = week_df.merge(los_x, on=['gameId', 'playId'], how='left')


In [ ]:
#This is for some future variables we use, standardized angle, etc

result_df['ToLeft'] = 0
result_df['ToLeft'] = result_df['playDirection'].apply(lambda x: 1 if x == 'left' else 0)

result_df['YardsFromOwnGoal'] = result_df[['yardlineSide','possessionTeam','yardlineNumber']].apply(lambda row:row.yardlineNumber if row.yardlineSide==row.possessionTeam else 50+(50-row.yardlineNumber), axis=1)

result_df['YardsFromOwnGoal'] = result_df[['YardsFromOwnGoal','yardlineNumber']].apply(lambda row:50 if row.yardlineNumber==50 else row.YardsFromOwnGoal, axis=1)

result_df['X_std'] = result_df[['ToLeft','x']].apply(lambda row:120-row.x-10 if row.ToLeft else row.x-10, axis=1)

result_df['Y_std'] = result_df[['ToLeft','y']].apply(lambda row:160/3-row.y if row.ToLeft else row.y, axis=1)

result_df['Dir_std_1'] = result_df[['ToLeft','dir']].apply(lambda row:row.dir+360 if (row.ToLeft and row.dir<90) else row.dir, axis=1)
result_df['Dir_std_1'] = result_df[['ToLeft','dir','Dir_std_1']].apply(lambda row:row.dir-360 if ((not row.ToLeft) and row.dir>270) else row.Dir_std_1, axis=1)
result_df['Dir_std_2'] = result_df[['ToLeft','Dir_std_1']].apply(lambda row:row.Dir_std_1-180 if row.ToLeft else row.Dir_std_1, axis=1)



# Assuming 'displayName' and 'ballCarrierDisplayName' are columns in result_df
result_df['ballcarrier'] = np.where(result_df['displayName'] == result_df['ballCarrierDisplayName'], 1, 0)


In [ ]:
#Need to put players.csv provided in order to get position by player
position = pd.read_csv('/content/drive/MyDrive/NFL DATA BOWL/players.csv')



In [ ]:
result_df = result_df.merge(position, on=['nflId'], how='left')
result_df = result_df.rename(columns={'displayName_x': 'displayName'})

# Which runs qualify for the SET Score

In [ ]:
def does_run_qualify(tracking_for_play):
    qualifies = 0

    filtered_df = tracking_for_play[(tracking_for_play['ballcarrier'] == 1) & (tracking_for_play['frameId'] == 1)]
    if not filtered_df.empty:
        offense_club = filtered_df['club'].iloc[0]
    else:
        print("NO TEAM GIVEN")
        return

    los = tracking_for_play[(tracking_for_play['displayName'] == 'football') & (tracking_for_play['frameId'] == 1)]
    los_x = los['x'].iloc[0]

    behind_los_frames = []
    for index, row in tracking_for_play.iterrows():
        if row['playDirection'] == 'right' and row['displayName'] == 'football':
            if row['x'] < los_x:
                behind_los_frames.append(row['frameId'])
        elif row['playDirection'] == 'left' and row['displayName'] == 'football':
            if row['x'] > los_x:
                behind_los_frames.append(row['frameId'])

    try:
        last_frame_behind_los = max(behind_los_frames)
    except ValueError:
        print("ODDITY", row['playId'])
        return

    behind_los_tracking = tracking_for_play[tracking_for_play['frameId'] <= last_frame_behind_los]

    tackles = behind_los_tracking[(behind_los_tracking['position'] == 'T') & (behind_los_tracking['frameId'] == 1)]
    tackles_y = tackles['y'].to_list()
    print(tackles_y)

    if len(tackles_y) == 1:
        guards = behind_los_tracking[(behind_los_tracking['position'] == 'G') & (behind_los_tracking['frameId'] == 1)]
        guards_y = guards['y'].to_list()
        tackles_y.append(guards_y)
        tackles_y = [item for sublist in tackles_y for item in (sublist if isinstance(sublist, list) else [sublist])]
    if len(tackles_y) == 0:
      print("ODDITY", row['playId'])
      return

    for index, row in behind_los_tracking.iterrows():
        if row['ballcarrier'] == 1:
            x, y, direction, speed = row["x"], row["y"], row["dir"], row["s"]
            adjusted_direction = np.mod(90 - direction, 360)
            x_comp = np.cos(np.radians(adjusted_direction))
            y_comp = np.sin(np.radians(adjusted_direction))
            v_x = speed * x_comp
            v_y = speed * y_comp
            mu_val_x = x + v_x * 0.5
            mu_val_y = y + v_y * 0.5
            mu = [mu_val_x, mu_val_y]
            min_tackle_value = min(tackles_y)
            max_tackle_value = max(tackles_y)
            is_between = min_tackle_value <= mu_val_y <= max_tackle_value

            if not is_between:
                qualifies = 1
                break

    return qualifies

# Iterate over games and plays
def check_all_runs(results_df):
    results = []

    for game_id in results_df['gameId'].unique():
        game_data = results_df[results_df['gameId'] == game_id]

        for play_id in game_data['playId'].unique():
            play_data = game_data[game_data['playId'] == play_id]
            qualifies = does_run_qualify(play_data)
            results.append({'gameId': game_id, 'playId': play_id, 'Qualifies': qualifies})

    return pd.DataFrame(results)

#final_results_df is going to be the runs which qualify for the score
final_results_df = check_all_runs(result_df)

In [ ]:
result_df = result_df.merge(final_results_df, on=['gameId', 'playId'])
#make a new dataframe of all the qualified runs
qualified = result_df[result_df['Qualifies']==1]

In [ ]:
#determine which direction the runner was intending to go
def left_or_right_run_intention(week7_tracking_runs_qualified, game_id, play_id):

    tracking_for_play = week7_tracking_runs_qualified[(week7_tracking_runs_qualified['gameId'] == game_id) & (week7_tracking_runs_qualified['playId'] == play_id)]

    threshold_tracking = tracking_for_play[(tracking_for_play['frameId'] <= 25) & (tracking_for_play['ballcarrier'] == 1)]

    ballcarrier_dirs = []

    for index, row in threshold_tracking.iterrows():
        ballcarrier_dirs.append(row['Dir_std_2'])

    left_count = sum(0 <= angle <= 90 for angle in ballcarrier_dirs)
    right_count = sum(90 < angle <= 180 for angle in ballcarrier_dirs)

    intended_direction = ""

    if left_count > right_count:
        intended_direction += "left"
    elif right_count > left_count:
        intended_direction += "right"

    return intended_direction

#determine who the edge setter is on each play 
from collections import Counter
def edge_setter_in_question(runs_that_qualify, game_id, play_id):

    tracking_for_play = runs_that_qualify[(runs_that_qualify['gameId'] == game_id) & (runs_that_qualify['playId'] == play_id)]

    los = tracking_for_play[(tracking_for_play['displayName'] == 'football') & (tracking_for_play['frameId'] == 1)]
    los_x = los['X_std'].iloc[0]

    positions = ["CB", "DB", "DE", "OLB"]

    ballcarrier_tracking_frames = tracking_for_play[tracking_for_play['ballcarrier'] == 1]
    target_tracking = tracking_for_play[(tracking_for_play['position'].isin(positions)) | (tracking_for_play['ballcarrier'] == 1)]

    ### GIVES US LOS FRAME
    mask = ballcarrier_tracking_frames['X_std'] > los_x

    first_valid_index = ballcarrier_tracking_frames[mask].first_valid_index()

    end_frame = ballcarrier_tracking_frames.loc[first_valid_index, 'frameId'] if first_valid_index is not None else None

    if end_frame is None:
        first_contact_index = ballcarrier_tracking_frames[ballcarrier_tracking_frames['event'] == 'first_contact'].first_valid_index()
        end_frame = ballcarrier_tracking_frames.loc[first_contact_index, 'frameId'] if first_contact_index is not None else ballcarrier_tracking_frames['frameId'].max()

    target_tracking = target_tracking[target_tracking['frameId'] <= end_frame]
    ballcarrier_tracking_filtered = ballcarrier_tracking_frames[ballcarrier_tracking_frames['frameId'] <= end_frame]

    ballcarrier_data = ballcarrier_tracking_filtered.copy()
    ballcarrier_data['x_comp'] = np.cos(np.radians(ballcarrier_data['Dir_std_2']))
    ballcarrier_data['y_comp'] = np.sin(np.radians(ballcarrier_data['Dir_std_2']))
    ballcarrier_data['v_x'] = ballcarrier_data['s'] * ballcarrier_data['x_comp']
    ballcarrier_data['v_y'] = ballcarrier_data['s'] * ballcarrier_data['y_comp']
    ballcarrier_data['mu_val_x'] = ballcarrier_data['X_std'] + ballcarrier_data['v_x'] * 0.5
    ballcarrier_data['mu_val_y'] = ballcarrier_data['Y_std'] + ballcarrier_data['v_y'] * 0.5

    closest_defenders = {}

    for frame_id in ballcarrier_data['frameId'].unique():
        # Ball carrier's target point for this frame
        bx = ballcarrier_data.loc[ballcarrier_data['frameId'] == frame_id, 'mu_val_x'].iloc[0]
        by = ballcarrier_data.loc[ballcarrier_data['frameId'] == frame_id, 'mu_val_y'].iloc[0]

        # Filter for defenders in the same frame
        frame_defenders = target_tracking[(target_tracking['frameId'] == frame_id) & (target_tracking['ballcarrier'] != 1)].copy()

        # Vectorized distance calculation
        frame_defenders['distance_to_target'] = np.sqrt((frame_defenders['X_std'] - bx) ** 2 + (frame_defenders['Y_std'] - by) ** 2)

        if not frame_defenders.empty:
            closest_defender = frame_defenders.loc[frame_defenders['distance_to_target'].idxmin()]
            closest_defenders[frame_id] = closest_defender['displayName']

        else:
            closest_defenders[frame_id] = None

    name_counter = Counter(closest_defenders.values())

    edge_rusher = name_counter.most_common(1)[0][0]
    return edge_rusher

In [ ]:
#apply the functions to qualified
def analyze_runs_in_qualified(qualified):
    results = []

    for game_id in qualified['gameId'].unique():
        game_data = qualified[qualified['gameId'] == game_id]

        for play_id in game_data['playId'].unique():
            play_data = game_data[game_data['playId'] == play_id]

            # Call your functions for each game and play
            run_intention = left_or_right_run_intention(qualified, game_id, play_id)
            edge_rusher = edge_setter_in_question(qualified, game_id, play_id)

            # Store the results
            results.append({'gameId': game_id, 'playId': play_id, 'RunIntention': run_intention, 'EdgeRusher': edge_rusher})

    return pd.DataFrame(results)

final_analysis_df = analyze_runs_in_qualified(qualified)

In [ ]:
#merge the new functions back into qualified
qualified = qualified.merge(final_analysis_df, on=['gameId', 'playId'])
#now qualified will be used going forward, I'd reccomend saving this to re open later, up to the user though

# Creation of the Overlap variable - the major aspect in the SET score


In [ ]:
#The overlap variable is the overlap between the Defenders Voronoi Regions and the ballcarrier's valuable space

In [ ]:


# FUNCTION TO CREATE FOOTBALL FIELD

def create_football_field():

    # Create a rectangle defined via an anchor point *xy* and its *width* and *height*
    rect = patches.Rectangle((0, 0), 120, 53.3, facecolor='green', alpha = 0.7, zorder=0)

    # Creating a subplot to plot our field on
    fig, ax = plt.subplots(1, figsize=(12, 6.33))

    # Adding the rectangle to the plot
    ax.add_patch(rect)

    # Plotting a line plot for marking the field lines
    #plt.plot([10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              #80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             #[0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3,
             # 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             #color='white', zorder = 0)

    # Creating the left end-zone
    left_end_zone = patches.Rectangle((0, 0), 10, 53.3, facecolor='blue', alpha=0.2, zorder=0)

    # Creating the right end-zone
    right_end_zone = patches.Rectangle((110, 0), 120, 53.3, facecolor='blue', alpha=0.2, zorder=0)

    # Adding the patches to the subplot
    ax.add_patch(left_end_zone)
    ax.add_patch(right_end_zone)

    # Setting the limits of x-axis from 0 to 120
    #plt.xlim(0, 120)

    # Setting the limits of y-axis from -5 to 58.3
    #plt.ylim(-5, 58.3)

    # Removing the axis values from the plot
    #plt.axis('off')

    # Plotting the numbers starting from x = 20 and ending at x = 110
    # with a step of 10
    for x in range(20, 110, 10):

        # Intializing another variable named 'number'
        number = x

        # If x exceeds 50, subtract it from 120
        if x > 50:
            number = 120 - x

        # Plotting the text at the bottom
        plt.text(x, 5, str(number - 10),
                 horizontalalignment='center',
                 fontsize=20,
                 color='white')

        # Plotting the text at the top
        plt.text(x - 0.95, 53.3 - 5, str(number - 10),
                 horizontalalignment='center',
                 fontsize=20,
                 color='white',
                 rotation=180)

    # Making ground markings
    for x in range(11, 110):
            ax.plot([x, x], [0.4, 0.7], color='white', zorder = 0)
            ax.plot([x, x], [53.0, 52.5], color='white', zorder = 0)
            ax.plot([x, x], [22.91, 23.57], color='white', zorder = 0)
            ax.plot([x, x], [29.73, 30.39], color='white', zorder = 0)

    # Returning the figure and axis
    return fig, ax


def calculate_voronoi_regions(player_coordinates):
    vor = Voronoi(player_coordinates)

    voronoi_regions = []

    for r in range(len(player_coordinates)):
        region = vor.regions[vor.point_region[r]]
        if not -1 in region:
            polygon_coords = [vor.vertices[i] for i in region]
            voronoi_regions.append(polygon_coords)

    return voronoi_regions
#these are the coordinates of the voronoi regions

In [ ]:
tracking = qualified.copy()
ballcarrier = tracking[tracking['ballcarrier']==1]
ballcarrier = ballcarrier.rename(columns={col: f'ballcarrier_{col}' for col in ballcarrier.columns})
# Create a subset of 'ballcarrier' with only the necessary columns
ballcarrier_subset = ballcarrier[['ballcarrier_gameId', 'ballcarrier_playId','ballcarrier_frameId', 'ballcarrier_s', 'ballcarrier_x']]

#merge this back into tracking as it allows for ballcarrier information to be in the same row as all players
tracking = tracking.merge(ballcarrier_subset, left_on=['gameId', 'playId', 'frameId'], right_on=['ballcarrier_gameId', 'ballcarrier_playId', 'ballcarrier_frameId'])


In [ ]:
#our metric cannot be run on frames where the ballcarrier speed is not above 0 
tracking['speed_metric'] = (tracking['ballcarrier_s'] > 0).astype(int)

# Sort DataFrame by playId and then by frameId
tracking = tracking.sort_values(by=['playId', 'frameId'])

# Identify where speed_metric goes from 1 to 0, this step is needed for ballcarrier's in motion pre snap
tracking['speed_drops_to_zero'] = (tracking['speed_metric'] == 0) & (tracking['speed_metric'].shift(1) == 1)

# Group by playId and find the first occurrence of speed dropping to 0 in each play
first_drop_indices = tracking.groupby('playId')['speed_drops_to_zero'].transform('idxmax')

# Create a mask to keep rows after the first drop to zero in each play
mask = tracking.index >= first_drop_indices

# Apply the mask to filter the DataFrame
tracking = tracking[mask]


In [ ]:
import numpy as np
# Calculate if the ballcarrier is past the LOS in each frame
tracking['is_past_threshold'] = np.where(
    ((tracking['playDirection'] == 'right') & (tracking['ballcarrier_x'] > tracking['los_x'])) |
    ((tracking['playDirection'] == 'left') & (tracking['ballcarrier_x'] < tracking['los_x'])),
    1, 0
)

# Use cummax to ensure that once the LOS is passed, it remains passed for all subsequent frames
tracking['past_threshold'] = tracking.groupby(['gameId', 'playId'])['is_past_threshold'].cummax()
tracking= tracking[tracking['past_threshold']==0]



In [ ]:

cmap = mpl.colormaps['viridis']

data_for_dataframe = []
cumulative_overlap_dict = {}

# THIS IS JUST THE FUNCTION - use of this function is later

def calculate_overlap_for_frame(frame_id, game_id, play_id, tracking, cumulative_dict):

    tracking_all = tracking[(tracking['playId'] == play_id)
                                 & (tracking['gameId'] == game_id)
                                 & (tracking['frameId'] == frame_id)]

    tracking_all = tracking_all.sort_values(by='club')
    tracking_all_no_football = tracking_all[tracking_all['club'] != 'football']

    # FOR GRAIENT COLORING ON PROB. DISTRIBUTION FUNCTION

    x, y = np.mgrid[0:120:1, 0:53.3:1]
    locations = np.dstack((x, y))
    x, y = np.mgrid[0:120:1, 0:53.3:1]
    fig, ax = create_football_field()


    ballcarrier = tracking_all[tracking_all['ballcarrier'] == 1]
    offensive_team = ballcarrier['club'].to_list()[0]



    offensive_data = tracking_all[(tracking_all['frameId'] == frame_id) & (tracking_all['club'] == offensive_team)]
    defense_data = tracking_all[(tracking_all['frameId'] == frame_id) & (tracking_all['club'] != offensive_team) & (tracking_all['club'] != 'football')]
    football_data = tracking_all[(tracking_all['frameId'] == frame_id) & (tracking_all['club'] == 'football')]

    offensive_x, offensive_y = offensive_data['x'], offensive_data['y']
    defense_x, defense_y = defense_data['x'], defense_data['y']
    football_x, football_y = football_data['x'], football_data['y']

    xy = tracking_all_no_football[['x','y']].values

    n_points = xy.shape[0]
    xy1 = xy.copy()
    xy1[:,1] = - xy[:,1]
    xy2 = xy.copy()
    xy2[:,1] = 320/3 - xy[:,1]
    xy3 = xy.copy()
    xy3[:,0] = 30 - xy[:,0]
    xy4 = xy.copy()
    xy4[:,0] = 220 - xy[:,0]
    xy = np.concatenate((xy, xy1, xy2, xy3, xy4), axis=0)

    voronoi_regions1 = calculate_voronoi_regions(xy)

    num_regions = len(voronoi_regions1)
    colors = ['red'] * 11 + ['lightblue'] * (11)

    polygons = [Polygon(region) for region in voronoi_regions1]

    plot = PatchCollection(polygons, facecolors = colors, alpha=0.35)

    ax.set_aspect('equal')

    for index, row in ballcarrier.iterrows():

        x, y, direction, speed = row["x"], row["y"], row["dir"], row["s"]
        ballcarrier_x = row['x']
        ballcarrier_y = row['y']

        speed_ratio = (speed**2)/(100)

        adjusted_direction = np.mod(90 - direction, 360)  # Apply the adjustment

        x_comp = np.cos(np.radians(adjusted_direction))
        y_comp = np.sin(np.radians(adjusted_direction))

        v_x = speed * x_comp
        v_y = speed * y_comp

        arrow = Arrow(x, y, v_x, v_y, width=1, color='white', zorder=2)

        scaling_matrix = np.diag([abs(v_x), abs(v_y)])

        if 215 < adjusted_direction < 315 or 45 < adjusted_direction < 135:

            rotation_matrix = np.array([[np.cos(np.radians(adjusted_direction - 90)), -np.sin(np.radians(adjusted_direction - 90))],
                                       [np.sin(np.radians(adjusted_direction - 90)), np.cos(np.radians(adjusted_direction - 90))]])

            rotation_matrix_T = rotation_matrix.T

            covariance_matrix = rotation_matrix @ scaling_matrix @ rotation_matrix_T

        else:

            rotation_matrix = np.array([[np.cos(np.radians(adjusted_direction)), -np.sin(np.radians(adjusted_direction))],
                                       [np.sin(np.radians(adjusted_direction)), np.cos(np.radians(adjusted_direction))]])

            rotation_matrix_T = rotation_matrix.T

            covariance_matrix = rotation_matrix @ scaling_matrix @ rotation_matrix_T
            covariance_matrix += np.eye(covariance_matrix.shape[0]) * 1e-8

        mu_val_x = x + v_x * 0.5
        mu_val_y = y + v_y * 0.5
        mu = [mu_val_x,mu_val_y]

        player_pdf = multivariate_normal(mu,covariance_matrix).pdf(locations)

        player_pdf_df = pd.DataFrame(data=player_pdf)

        player_pdf_values = player_pdf_df.values
        new_cmap = LinearSegmentedColormap.from_list('custom_cmap', ['green', 'green'])


        #normalized_player_pdf_values = (player_pdf_values - np.min(player_pdf_values)) / (np.max(player_pdf_values) - np.min(player_pdf_values))

        x, y = np.mgrid[0:120:1, 0:53.3:1]
        exact_contour_filled = ax.contourf(x, y, player_pdf_values, cmap=new_cmap, levels=1, zorder=2)
        viz_contour_filled = ax.contourf(x, y, player_pdf_values, cmap=new_cmap, levels=25, zorder=2)


        #football_scatter = ax.scatter(football_x, football_y, c='brown', edgecolors='white', alpha=0.75, linewidth=0.75, s=40, zorder=3)
        voronoi_player_dict = {}

        n_players = min(len(tracking_all_no_football['displayName']), 22)
        n_regions = min(len(voronoi_regions1), 22)

        for player, region in zip(tracking_all_no_football['displayName'][:n_players], voronoi_regions1[:n_regions]):
            voronoi_player_dict[player] = region

        defense_players_list = defense_data['displayName'].to_list()

        voronoi_player_dict_defense = {}

        for key, value in voronoi_player_dict.items():
            if key in defense_players_list:
                voronoi_player_dict_defense[key] = value

        voronoi_player_dict_defense_clean = {player: [array.tolist() for array in regions]
                                         for player, regions in voronoi_player_dict_defense.items()}

        values = voronoi_player_dict_defense_clean.values()
        list_vals = list(values)

        for verts in list_vals:
            for cord in verts:
                x, y = cord
                ax.scatter(x, y, c='white', s=10)


        contour_path = exact_contour_filled.collections[0].get_paths()[0]
        countour_vertices = contour_path.vertices
        countour_vertices_list = countour_vertices.tolist()

        countour_cleaned_vertices = []

        for pair in countour_vertices_list:
            if len(str(pair[0])) > 5 or len(str(pair[1])) > 5:
                countour_cleaned_vertices.append(pair)

        countour_polygon = PolygonArea(countour_cleaned_vertices)
        #print(countour_polygon.area)
        data_for_dataframe = []
        for key, value in voronoi_player_dict_defense_clean.items():
            voronoi_polygon = PolygonArea(value)
            intersection = countour_polygon.intersection(voronoi_polygon)

            overlap_area = intersection.area
            overlap_percentage = (overlap_area / countour_polygon.area) * 100  # Assuming percentage calculation

            # Update cumulative_dict
            cumulative_dict[key] = cumulative_dict.get(key, 0) + overlap_percentage

            # Prepare data for the DataFrame
            data_for_dataframe.append([game_id, play_id, frame_id, key, overlap_percentage])

    return data_for_dataframe

In [ ]:
#Make sure to select the game_id in which you want the SET scores for

desired_segment_df = tracking[tracking['game_id']==game_id]

from concurrent.futures import ThreadPoolExecutor, as_completed

def process_frame(frame_id, game_id, play_id, tracking):
    try:
        frame_data = calculate_overlap_for_frame(frame_id, game_id, play_id, tracking, cumulative_overlap_dict)
        return frame_data
    except Exception as e:
        return {
            'game_id': game_id,
            'play_id': play_id,
            'frame_id': frame_id,
            'error_message': str(e),
            'type': 'error'
        }

# Assuming all_data is a list
all_data = []

# Adjust the number of workers based on your system's capabilities
num_workers = 4

with ThreadPoolExecutor(max_workers=num_workers) as executor:
    futures = []
    total_frames = 0

    #MAKE SURE TO ENTER THE game_id 
        single_game = tracking[tracking['gameId'] == game_id]

        for play_id in single_game['playId'].unique():
            single_play = single_game[single_game['playId'] == play_id]
            frame_max = single_play['frameId'].max()
            frame_min = single_play['frameId'].min()

            for frame_id in range(frame_min, frame_max + 1):
                futures.append(executor.submit(process_frame, frame_id, game_id, play_id, tracking))
                total_frames += 1

    completed_frames = 0
    # Incremental collection of results with progress tracking
    for future in as_completed(futures):
        result = future.result()
        completed_frames += 1

        if isinstance(result, list):
            all_data.extend(result)
        else:
            all_data.append(result)

        print(f"Completed {completed_frames}/{total_frames} frames")

    print("All frames processed")

In [ ]:
column_names = ['gameId', 'playId', 'frameId', 'playerName', 'overlapPercentage']
overlap_df = pd.DataFrame(all_data, columns=column_names)

In [ ]:

# Assuming overlap_df and qualified are already defined
#Make sure qualified is the same as the point earlier where I said to save it

# Step 1: Check data types
print("Data types in overlap_df:")
print(overlap_df[['gameId', 'playId', 'frameId', 'playerName']].dtypes)
print("\nData types in qualified:")
print(qualified[['gameId', 'playId', 'frameId', 'displayName']].dtypes)

# Step 2: Convert data types if necessary
# Example: If 'gameId' is int in one DataFrame and string in another, convert both to string
overlap_df['gameId'] = combined_df['gameId'].astype(str)
qualified['gameId'] = qualified['gameId'].astype(str)

# Do the same for 'playId', 'frameId', and rename 'displayName' to 'playerName' in 'qualified'
overlap_df['playId'] = combined_df['playId'].astype(str)
qualified['playId'] = qualified['playId'].astype(str)

overlap_df['frameId'] = combined_df['frameId'].astype(str)
qualified['frameId'] = qualified['frameId'].astype(str)

qualified_renamed = qualified.rename(columns={'displayName': 'playerName'})

# Step 3: Perform the merge
merged_df = pd.merge(overlap_df, qualified_renamed, how='left', on=['gameId', 'playId', 'frameId', 'playerName'])
#SET score only applies to 3 positions, OLB, CB, and DE
filtered_df = merged_df[merged_df['position'].isin(['OLB', 'CB', 'DE'])]




# Other features that are part of the SET score

In [ ]:
# Convert 'overlapPercentage' to numeric
filtered_df['overlapPercentage'] = pd.to_numeric(filtered_df['overlapPercentage'], errors='coerce')

# Calculate the cumulative overlap
cumulative_overlap = filtered_df.groupby(['gameId', 'playId', 'playerName'])['overlapPercentage'].sum().reset_index()
cumulative_overlap = cumulative_overlap.rename(columns={'overlapPercentage': 'cumulativeOverlap'})

# Rank players within each gameId and playId based on the cumulativeOverlap
cumulative_overlap['overlapRank'] = cumulative_overlap.groupby(['gameId', 'playId'])['cumulativeOverlap'].rank(ascending=False, method='first')

# Merge the rankings back into the original filtered_df
# Merge, ensuring each player in each play gets their new rank
filtered_df = pd.merge(filtered_df, cumulative_overlap[['gameId', 'playId', 'playerName', 'cumulativeOverlap', 'overlapRank']], on=['gameId', 'playId', 'playerName'], how='left')


In [ ]:
#bring back qualified again and assign it to the new variable, nonf
nonf = qualified

In [ ]:
#this is used to determine the intended direction of the ballcarrier while also noting the endzone they're approaching

nonf['right_left'] = nonf.apply(lambda x: 1 if x['playDirection'] == 'right' and x['RunIntention'] == 'left' else 0, axis=1)
nonf['right_right'] = nonf.apply(lambda x: 1 if x['playDirection'] == 'right' and x['RunIntention'] == 'right' else 0, axis=1)
nonf['left_right'] = nonf.apply(lambda x: 1 if x['playDirection'] == 'left' and x['RunIntention'] == 'right' else 0, axis=1)
nonf['left_left'] = nonf.apply(lambda x: 1 if x['playDirection'] == 'left' and x['RunIntention'] == 'left' else 0, axis=1)

In [ ]:
#these steps are used to calculate another part of our SET score, which is angle change of the ballcarrier

# Step 1: Create a DataFrame for ballcarrier
ballcarrier_df = nonf[nonf['ballcarrier'] == 1].copy()

# Step 2: Calculate ballcarrier_direction_1 and ballcarrier_direction_2
# ballcarrier_direction_1
ballcarrier_df['ballcarrier_direction_1'] = ballcarrier_df.apply(
    lambda row: row.dir + 360 if (row.ToLeft and row.dir < 90) else row.dir,
    axis=1)

# Adjust ballcarrier_direction_1
ballcarrier_df['ballcarrier_direction_1'] = ballcarrier_df.apply(
    lambda row: row.dir - 360 if ((not row.ToLeft) and row.dir > 270) else row.ballcarrier_direction_1,
    axis=1)

# ballcarrier_direction_2
ballcarrier_df['ballcarrier_direction_2'] = ballcarrier_df.apply(
    lambda row: row.ballcarrier_direction_1 - 180 if row.ToLeft else row.ballcarrier_direction_1,
    axis=1)

#ballcarrier y coordinate in 5 frames

ballcarrier_df['y_in_5_frames'] = ballcarrier_df.groupby(['gameId', 'playId'])['y'].shift(-5)
ballcarrier_df['direction_in_5_frames'] = ballcarrier_df.groupby(['gameId', 'playId'])['ballcarrier_direction_2'].shift(-5)
ballcarrier_df['difference'] = ballcarrier_df['direction_in_5_frames']-ballcarrier_df['ballcarrier_direction_2']

def calculate_good_job(row):
    if (row['left_left'] == 1 or row['right_left'] == 1) and row['difference'] > 0:
        return 1  # Set to 1 for positive difference in left left/right left
    elif (row['left_right'] == 1 or row['right_right'] == 1) and row['difference'] < 0:
        return 1  # Set to 1 for negative difference in left right/right right
    else:
        return 0  # Default case

# Apply the function to create the new column
ballcarrier_df['good_job'] = ballcarrier_df.apply(calculate_good_job, axis=1)

#Merge the new columns back into the original DataFrame 'nonf'
# Selecting only the relevant columns for merge
ballcarrier_df = ballcarrier_df.add_prefix('ballcarrier_')


columns_to_merge = ['ballcarrier_gameId', 'ballcarrier_playId', 'ballcarrier_frameId', 'ballcarrier_x', 'ballcarrier_y','ballcarrier_y_in_5_frames','ballcarrier_difference','ballcarrier_direction_in_5_frames','ballcarrier_ballcarrier_direction_2','ballcarrier_good_job']

nonf = pd.merge(nonf, ballcarrier_df[columns_to_merge],
                left_on=['gameId', 'playId', 'frameId'],
                right_on=['ballcarrier_gameId', 'ballcarrier_playId', 'ballcarrier_frameId'],
                how='left')

# Optionally, you can drop the redundant gameId, playId, and frameId columns from ballcarrier_df if they exist
nonf.drop(columns=['ballcarrier_gameId', 'ballcarrier_playId', 'ballcarrier_frameId'], inplace=True)




In [ ]:
#this step is another feature of our score
#it has to do with whether or not the ballcarrier ends up outside of the edge defender
#if the ballcarrier ends up outside, we set the score to zero, as the edge was not set properly

def is_inside(row):
    # Check the play direction based on specific columns
    if row.left_right == 1 or row.right_left == 1:
        return 1 if row.y < row.ballcarrier_y_in_5_frames else 0
    elif row.left_left == 1 or row.right_right == 1:
        return 1 if row.y > row.ballcarrier_y_in_5_frames else 0
    else:
        return 0  # Default case if no play direction is specified

# Apply the function to each row in nonf
nonf['is_inside'] = nonf.apply(is_inside, axis=1)


In [ ]:
#bring in the tackles.csv provided

tackles = pd.read_csv('path here',None)

In [ ]:

# Assuming nonf and tackles are your DataFrames
# Step 1: Check and convert data types
# Convert gameId, playId, and nflId to string in both DataFrames (formatting gets messed up without this step)
nonf['gameId'] = nonf['gameId'].astype(str)
nonf['playId'] = nonf['playId'].astype(str)
nonf['nflId'] = nonf['nflId'].astype(str)

tackles['gameId'] = tackles['gameId'].astype(str)
tackles['playId'] = tackles['playId'].astype(str)
tackles['nflId'] = tackles['nflId'].astype(str)

# Step 2: Perform the merge, this will allow us to see which player made the tackle
nonf = pd.merge(nonf, tackles[['gameId', 'playId', 'nflId', 'tackle']],
                on=['gameId', 'playId', 'nflId'],
                how='left')

# Replace NaN in the 'tackle' column with 0
nonf['tackle'].fillna(0, inplace=True)

# Convert 'tackle' to integer
nonf['tackle'] = nonf['tackle'].astype(int)


In [ ]:
#Now the dataframe only contains the players with the highest overlap on each play, we only give one score per play
only1 = filtered_df[filtered_df['overlapRank']==1]

In [ ]:
#make sure the week variable is still correct
only1['week'] = week

In [ ]:


# Assuming only1 and nonf are your DataFrames

# Step 1: Check data types and convert if necessary
# Convert gameId, playId, and frameId to string if they are not already
only1['gameId'] = only1['gameId'].astype(str)
only1['playId'] = only1['playId'].astype(str)
only1['frameId'] = only1['frameId'].astype(str)
nonf['gameId'] = nonf['gameId'].astype(str)
nonf['playId'] = nonf['playId'].astype(str)
nonf['frameId'] = nonf['frameId'].astype(str)

# Also ensure playerName and displayName are of the same type
only1['playerName'] = only1['playerName'].astype(str)
nonf['displayName'] = nonf['displayName'].astype(str)

# Step 2: Perform the merge
merged_df = pd.merge(only1, nonf, left_on=['gameId', 'playId', 'frameId', 'playerName'],
                     right_on=['gameId', 'playId', 'frameId', 'displayName'], how='left')




In [ ]:
#this is the change in ballcarrier angle standarized into a decimal
merged_df['standardized_difference'] = abs(merged_df['ballcarrier_difference']/90)


# Convert 'overlapPercentage' to numeric, coerce any non-numeric values to NaN
merged_df['overlapPercentage'] = pd.to_numeric(merged_df['overlapPercentage'], errors='coerce')

# Create the 'is_overlap' column based on whether 'overlapPercentage' is greater than 0
# NaN values will be treated as False in this comparison
merged_df['is_overlap'] = (merged_df['overlapPercentage'] > 0).astype(int)




# Creation of the SET metric

In [ ]:
#this is where the set score is created
merged_df['set_score'] = ((merged_df['overlapPercentage']/100) + merged_df['standardized_difference'])/2 * merged_df['ballcarrier_good_job']

In [ ]:
#this is a metric to see whether or not the edge setter was successful in setting the edge, it's binary
only_overlap = merged_df[merged_df['is_overlap']==1]
only_overlap['successful'] = only_overlap.apply(lambda row: 1 if row['is_overlap'] == 1 and row['set_score'] != 0 else 0, axis=1)


In [ ]:
# Perform the groupby and aggregation
stats_df = only_overlap.groupby(['gameId', 'playId', 'playerName']).agg({
    'set_score': ['sum', 'mean'],
    'position_x': 'first',
    'expectedPointsAdded_x': 'first',
    'passProbability_x': 'first',
    'playResult_x': 'first',
    'overlapPercentage':'first',
    'standardized_difference':'first',
    'tackle':'first',
    'successful':'first'
}).reset_index()

# Flatten the MultiIndex columns
stats_df.columns = ['_'.join(col).strip('_') for col in stats_df.columns.values]

# Rename the columns for clarity
stats_df.rename(columns={
    'set_tackle_sum': 'cumulative_set_tackle',
    'set_tackle_mean': 'average_set_tackle',
    'position_x_first': 'position_x',
    'epa_first': 'expectedPointsAdded_x',
    'pass_probability_x_first': 'passProbability_x',
    'playResult_x_first': 'playResult_x',
    'successful_first':'successful'
}, inplace=True)




In [ ]:
#This step takes the old set_score and adds in the tackle factor, to award players for making the play


# Function to modify set_score_sum and set_score_mean based on tackle_first
def modify_scores(row):
    if row['tackle_first'] == 1:
        row['set_score_sum_incl_tackle'] = row['set_score_sum'] + 0.5
        row['set_score_mean_incl_tackle'] = row['set_score_mean'] + 0.5
    else:
        row['set_score_sum_incl_tackle'] = row['set_score_sum']
        row['set_score_mean_incl_tackle'] = row['set_score_mean']
    return row

# Apply the function to the DataFrame
stats_df = stats_df.apply(modify_scores, axis=1)




In [ ]:
# Identify categorical and numerical columns
categorical_cols = stats_df.select_dtypes(include=['object', 'category']).columns
numerical_cols = stats_df.select_dtypes(include=['number']).columns

# Define aggregation logic
aggregation_logic = {col: 'first' for col in categorical_cols if col != 'playerName'}  # Keep first value for categorical columns, except 'playerName'
aggregation_logic.update({col: 'mean' for col in numerical_cols if col not in ['cumulative_set_tackle', 'set_score_sum_incl_tackle', 'set_score_mean_incl_tackle']})  # Average for numerical, except specified columns

aggregation_logic['set_score_sum_incl_tackle'] = 'sum'  # Sum for 'set_score_sum_incl_tackle'
aggregation_logic['set_score_mean_incl_tackle'] = 'mean'  # Mean for 'set_score_mean_incl_tackle'

# Add a column to count the number of plays per 'gameId' and 'playerName' group
stats_df['num_plays'] = stats_df.groupby(['gameId', 'playerName'])['playId'].transform('nunique')
aggregation_logic['num_plays'] = 'first'  # Keep the first value of num_plays, which is the same for all rows in the group

# Group by 'gameId' and 'playerName', then apply aggregation
grouped_df = stats_df.groupby(['gameId', 'playerName'], as_index=False).agg(aggregation_logic)

# Display the DataFrame
grouped_df = grouped_df.sort_values(by='set_score_sum_incl_tackle', ascending=False)



In [ ]:
#calculate number of successful plays
grouped_df['successful plays'] = grouped_df['num_plays'] * grouped_df['successful']

In [ ]:
#save grouped_df as a csv and view it, the SET score should be complete
#the tackle SET score is the version we chose to use in our work

# Note these cells are if you are looping through multiple games, percentile doesn't make sense if there isn't a large enough sample size


In [ ]:

# Assuming grouped_df is your DataFrame

# Function to scale percentile rank to 1-99 range
def scale_percentile(series):
    percentile_rank = series.rank(pct=True)  # Calculate percentile rank
    scaled_percentile = percentile_rank * 99  # Scale to 99 instead of 100
    return scaled_percentile.round().clip(lower=1)  # Round and clip to ensure the range is 1 to 99

# Calculate scaled percentile for 'cumulative_set_tackle'
grouped_df['cumulative_set_tackle_percentile'] = (grouped_df['set_score_sum_incl_tackle'].rank(pct=True) * 100).round()

# Calculate scaled percentile for 'average_set_tackle'
grouped_df['average_set_tackle_percentile'] = (grouped_df['set_score_mean_incl_tackle'].rank(pct=True) * 100).round()

# Calculate the absolute value of 'standardized_difference_first'
grouped_df['standardized_difference_first_abs'] = grouped_df['standardized_difference_first'].abs()

# Calculate scaled percentile for 'overlapPercentage_first'
grouped_df['overlapPercentage_percentile'] = scale_percentile(grouped_df['overlapPercentage_first'])

# Calculate scaled percentile for the absolute value of 'standardized_difference_first'
grouped_df['standardized_difference_percentile'] = scale_percentile(grouped_df['standardized_difference_first_abs'])






In [ ]:


# Assuming grouped_df is your DataFrame
# Assuming the percentile column for cumulative percent is named 'cumulative_set_tackle_percentile'

# Filter and get top 10 for CB
top10_CB = grouped_df[grouped_df['position_x'] == 'CB'].nlargest(25, 'cumulative_set_tackle_percentile')

# Filter and get top 10 for OLB
top10_OLB = grouped_df[grouped_df['position_x'] == 'OLB'].nlargest(25, 'cumulative_set_tackle_percentile')

# Filter and get top 10 for DE
top10_DE = grouped_df[grouped_df['position_x'] == 'DE'].nlargest(25, 'cumulative_set_tackle_percentile')




In [ ]:
#This is when looping through multiple games